# TODOs
- [ ] Runtimes with various input size and different optimization flags
> Create a timing infrastructure to get runtimes for a suitable set of input sizes (and inputs). Push sizes to the limit (i.e., until the execution takes minutes or even hours). Consider different compiler flags (but always with optimization).
- [ ] Cost Analysis (float ops, float comp, int ops, int comp)
> Cost analysis: Select a cost measure (usually flop count but may include div, sin, ...) and determine the cost. Briefly say how you did it (e.g., counted in code, had to instrument parts, ...).
- [ ] Performance Plot
> Create a performance plot. Put it as plot0 (or plot0a, plot0b, ...) into svn in the same directory as this document. As you perform optimizations later, include one line for each relevant optimization.
- [ ] Bottlenecks by profiling tool
> Identify performance bottlenecks in your code by profiling. This will likely involve instrumenting your code (standard profilers may not be fine grain enough). Focus on those parts when optimizing. The bottlenecks may change during optimization. Understand the cost and performance of the bottlenecks to set expectations right.

In [1]:
import re
import numpy as np    
import pandas as pd

general_log = "./general_metrics.txt"
cache_log = './cache_log.txt'
flops_log = './flops_log.txt'

In [11]:
def extract(log, metric):
    res = []
    with open(log, 'r') as f:
        file = f.read()
    try:
        if metric=='runtime(sec)':
            res = re.findall("(.*) seconds time elapsed", file)
        elif metric=='cc_int_ops':
            res = re.findall("cc_int_ops:(.*), cc_int_comp", file)
        elif metric=='cc_int_comp':
            res = re.findall("cc_int_comp:(.*), cc_fl_ops", file)
        elif metric=='cc_fl_ops':
            res = re.findall("cc_fl_ops(.*), cc_fl_comp", file)
        elif metric=='cc_fl_comp':
            res = re.findall("cc_fl_comp:(.*)", file)
        elif metric=='ll_int_ops':
            res = re.findall("ll_int_ops:(.*), ll_int_comp:", file)
        elif metric=='ll_int_comp':
            res = re.findall("ll_int_comp:(.*)", file)
        elif metric=='flops':
            res = re.findall("#fl ops: (.*)", file)
        elif metric=='flcomp':
            res = re.findall("#fl comparisons: (.*)", file)
        elif metric=='intops':
            res = re.findall("#int ops: (.*)", file)
        elif metric=='intcomp':
            res = re.findall("#int comparisons: (.*)", file)
        elif metric=='cycles':
            res = re.findall("#cycles: (.*)", file)
        elif metric=='flops/cycles':
            res = re.findall("(.*)flops / cycles", file)
        elif metric=='instructions':
            res = re.findall("(.*)   instructions:u", file)
        elif metric=='branches':
            res = re.findall("(.*)     branches:u", file)
        elif metric=='branch-misses':
            res = re.findall("branch-misses:u           #(.*) of all branches", file)
        elif metric=='page-faults':
            res = re.findall("(.*)  page-faults:u", file)
        elif metric=='cycles(perf)':
            res = re.findall("(.*)      cycles:u", file)
        elif metric=='flops(perf)':
            re.findall("(.*) r5301c7", file)
    except Exception:
        return 
    return res  

def init_table(vertice, edge):
    ve = []
    for v in vertice:
        for e in edge:
            if e < v*10 and e >= v:
                ve.append([v,e])
    return pd.DataFrame(ve, columns=['#V', '#E'])

def update_table(df, log, data, header):
    for i, x in enumerate(df['#V']):
        for j, y in enumerate(df['#E']):
            for l in header:
                data[l] = extract(log, l)
                if len(data[l])==len(df):
                    df[l] = data[l]
                else:
                    print(f"fail to add {l} to the table.")

In [10]:
from itertools import product

label=['runtime(sec)',
       'flops','flcomp','intops','intcomp', 'flops/cycles',
      #'cc_int_ops', 'cc_int_comp', 'cc_fl_ops', 'cc_fl_comp', 'll_int_ops', 'll_int_comp',
      'instructions', 'cycles', 'cycles(perf)', 
      'branches', 'branch-misses', 'page-faults']
cache=['L1-dcache-loads','L1-dcache-load-misses','L1-dcache-stores',
       'LLC-loads','LLC-load-misses','LLC-stores','LLC-store-misses',
       'dTLB-loads','dTLB-load-misses','dTLB-stores','dTLB-store-misses',
       'iTLB-loads', 'iTLB-load-misses']
flops=['flops(perf)']

vertice = [10,100,1000,2000,5000,8000,10000]
edge = [10,100,500,1000,2000,5000,8000,10000,20000,50000]

In [14]:
# results with flag -O3
data = dict()
res = init_table(vertice, edge)
res = res[:-1]
res

,#V,#E
0,10,10
1,100,100
2,100,500
3,1000,1000
4,1000,2000
5,1000,5000
6,1000,8000
7,2000,2000
8,2000,5000
9,2000,8000


In [15]:
update_table(res, general_log, data, label)
res

,#V,#E,runtime(sec),flops,flcomp,intops,intcomp,flops/cycles,instructions,cycles,cycles(perf),branches,branch-misses,page-faults
0,10,10,0.010035947,71,15,50,126,0.00664622,"43,734,003",39420.9,"27,406,913","7,738,926",0.70%,420
1,100,100,0.160081721,1278,275,776,2007,0.00547562,"1,032,889,526",791874,"576,709,917","175,378,523",0.32%,430
2,100,500,0.686663846,12156,7819,13649,36298,0.0187632,"4,696,918,569",3.72656e+06,"2,490,286,717","788,318,319",0.46%,451
3,1000,1000,8.242615681,13296,2851,8002,20783,0.000961764,"64,372,965,223",4.67183e+07,"30,047,979,584","10,859,438,953",0.09%,495
4,1000,2000,15.706050421,68545,31960,58659,160084,0.00356626,"130,012,123,572",8.95189e+07,"57,834,703,517","22,097,969,079",0.11%,547
5,1000,5000,38.363303375,195773,132635,228933,594329,0.00520455,"323,303,307,143",2.21281e+08,"140,851,750,932","55,307,647,959",0.13%,802
6,1000,8000,62.502320174,302344,240501,412317,1038420,0.00553681,"518,710,449,596",3.6006e+08,"228,356,545,273","88,826,849,381",0.15%,976
7,2000,2000,29.968098914,26970,5765,16054,41925,0.000534638,"250,554,518,237",1.69674e+08,"109,473,398,766","42,029,257,233",0.05%,567
8,2000,5000,74.865500267,202701,109035,192922,516866,0.00240122,"623,087,866,977",4.25419e+08,"272,429,590,003","106,254,362,155",0.07%,804
9,2000,8000,117.836783339,347583,223132,386353,1009192,0.00291904,"993,016,470,658",6.73598e+08,"429,680,535,192","170,070,058,716",0.08%,"1,097"


In [17]:
res.to_excel('result.xlsx', sheet_name='sheet1', index=False) 